In [1]:
# mount the content of the folder called AIR_Project in google drive here

from google.colab import drive
drive.mount('/content/drive')

!ln -s "/content/drive/My Drive/AIR_Project" "/content/AIR_Project"


Mounted at /content/drive


In [2]:
import AIR_Project.dataset_preparation as dataset_preparation

Train dataset loaded successfully with 82022 records.
Val dataset loaded successfully with 5592 records.
Test dataset loaded successfully with 5593 records.


In [3]:
# Load the datasets
df_test_data = dataset_preparation.get_test_dataset()

In [4]:
!pip install --quiet  tqdm     #for progress bars
!pip install --quiet transformers # for t5 model
!pip install --quiet tokenizers  #tokenizers from HuggingFace
!pip install --quiet sentencepiece #subword tokenizer used by T5
!pip install --quiet pytorch-lightning # pytorch wrapper
!pip install --quiet torchtext # text utilities

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.8 MB/s eta 0:00:00


In [5]:
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
from transformers import (
    PegasusForConditionalGeneration,
    PegasusTokenizer
)

In [6]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"

In [7]:
tokenizer = PegasusTokenizer.from_pretrained('alireza7/TRANSFORMER-persian-base-PN-summary',model_max_length=512)
model = PegasusForConditionalGeneration.from_pretrained('alireza7/TRANSFORMER-persian-base-PN-summary')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

In [8]:
class TextSummarizationDataset(Dataset):
    def __init__(self, tokenizer, split='train', max_len_inp=512,max_len_out=96):
        self.split = split

        self.article_column = "article"
        self.summary_column = "summary"

        if self.split == 'train':
          self.data = df_train_data
        elif self.split == 'val':
          self.data = df_val_data
        elif self.split == 'test':
          self.data = df_test_data

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim]

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows()): # Iterating over the dataframe
            article,summary = val[self.article_column],val[self.summary_column]

            input_ = article # Input format for text summarization tasks
            target = summary # Output format we require

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input,padding='max_length',
                truncation = True,return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output,padding='max_length',
                truncation = True,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [9]:
test_dataset = TextSummarizationDataset(tokenizer, 'test')

5593it [00:19, 292.05it/s]


In [10]:
model = model.to(device)

In [13]:
articles = []
references = []
hypotheses = []

num = 10

for i in tqdm(range(num)):
    item = test_dataset[i]

    article = tokenizer.decode(item['source_ids'], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    summary = tokenizer.decode(item['target_ids'], skip_special_tokens=True,clean_up_tokenization_spaces=True)

    articles.append(article)
    references.append(summary)

    encoding = tokenizer.encode_plus(article,max_length =512,padding='max_length',
                                 truncation = True,
                                 return_tensors="pt").to(device)
    input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=72, # How long the generated summary should be
      early_stopping=True,
      num_beams=5,
      num_return_sequences=1
    )

    for beam_output in beam_outputs:
      predicted = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      hypotheses.append(predicted)

      print(f'article : {article}')
      print(f'true summary : {summary}')
      print(f'predicted summary: {predicted}')
      print('-'*50)

 10%|█         | 1/10 [00:00<00:07,  1.19it/s]

article : مدیر شرکت ملی پخش فرآورده های نفتی منطقه سبزوار به خبرنگار شانا، گفت: مصرف نفت کوره این منطقه به دلیل توسعه شبکه گاز طبیعی در سه ماه نخست امسال، با ۵۸ درصد کاهش نسبت به مدت مشابه سال پیش، به ۳۷۰ هزار لیتر رسیده است. [n] عیسی افتخاری سهم بنزین مصرفی این منطقه را نیز در بازه زمانی یاد شده ۳۸ میلیون و ۹۷۵ هزار لیتر اعلام و اظهار کرد: این مقدار از افزایش چهار درصدی مصرف این سوخت در قیاس با بهار پارسال حکایت دارد. [n] به گفته وی، در ۹۳ روز نخست امسال، ۴۶ میلیون و ۲۳۶ هزار لیتر نفت گاز در منطقه سبزوار مصرف شده که نسبت به مدت مشابه سال پیش، با افزایش نیم درصدی همراه شده است. [n] مدیر شرکت ملی پخش فرآورده های نفتی منطقه سبزوار همچنین، از مصرف سه میلیون و ۹۵۹ هزار لیتر نفت سفید در این منطقه خبر داد و افزود: مصرف این سوخت در قیاس با سه ماه نخست پارسال، ۲۶ درصد کاهش یافته است. [n] افتخاری تصریح کرد: مجموع مصرف انواع فرآورده نفتی منطقه سبزوار در نخستین فصل امسال، با یک درصد کاهش نسبت به مدت مشابه پارسال، به ۸۹ میلیون و ۵۴۱ هزار لیتر رسیده است. [n] وی یادآور شد: در ۹۳ روز نخست امسال، مصرف

 20%|██        | 2/10 [00:02<00:11,  1.45s/it]

article : به گزارش خبرگزاری مهر به نقل از معاونت علمی و فناوری، پیش بینی سن بر اساس ویژگی های مورفولوژی تصاویر MRI ساختاری در افراد سالم عنوان چالشی است که توسط آزمایشگاه ملی نقشه برداری مغز برگزار می شود تا روشی نوین برای پیش بینی سن افراد ۲۰ تا ۷۰ سال ارائه شود. [n] این رویداد نخستین چالش ملی مبتنی بر داده های بیوبانک نقشه برداری مغز ایران است. داده های علمی بیوبانک نقشه برداری مغز ایران در بستر محاسبات سریع، برای نخستین بار محققان را به چالش می کشد. [n] این چالش در راستای رسالت آموزشی و ترویجی آزمایشگاه و با هدف ارائه روش نوین در پیش بینی سن افراد بین ۲۰ تا ۷۰ سال با استفاده از تصاویر MRI طراحی شده است. [n] این چالش با داده های موجود در بیوبانک نقشه برداری مغز ایران و با به کارگیری سرور محاسبات سریع آزمایشگاه ملی نقشه برداری مغز، در دو مرحله غیر حضوری و حضوری برگزار می شود و محققان با آنالیز این داده ها به رقابت می پردازند. [n] ثبت نام چالش ۵ مهر ماه آغاز شده است. شرکت کنندگان پس از ثبت نام می توانند در تاریخ ۱۹ مهر داده های لازم را که بر وبسایت آزمایشگاه بارگذاری می شود، دریافت کنن

 30%|███       | 3/10 [00:04<00:10,  1.47s/it]

...

article : چاپ کتاب در همه جای دنیا کار سختی است اما در بعضی کشورها مراحل پیچیده اداری، ممیزی ها و پروسه های دیگر مزید بر علت می شود و این دشواری را دو چندان می کند. [n] برای چاپ یک کتاب باید مراحلی مانند تایپ، حروف چینی، صفحه آرایی، ویراستاری، نمونه خوانی، صحافی و طراحی جلد را انجام داد و بعد از گذشتن از تمام این مراحل وارد مرحله پخش شد، مرحله ای که به سبب وجود برخی انحصار طلبی ها مشکلات خاص خودش را دارد. [n] حوزه نشر به عنوان یکی از مهمترین حوزه های فرهنگی که با آگاهی و دانش جامعه ارتباط مستقیم دارد، همواره نیازمند توجه و رسیدگی مسوولان است. [n] به گفتهکی از ناشران زاهدانی؛ رعایت نشدن حقوق تولیدکنندگان در همه حوزه ها به ویژه حوزه فرهنگ، آسیب زننده است و امروز بزرگترین مشکل ناشران، مردمی هستند که حتی کتاب های کپی هم نمی خوانند و دانلودشان نیز نمی کنند. [n] تمامی تلاش نویسندگان و ناشران برای اندک مردمی است که هنوز کتاب می خوانند. [n] از زمانیکه شبکه های اجتماعی فراگیر شده بسیاری از مردم ناآگاهانه نسخه های دیجیتال کتاب ها را در سایت ها و کانال هایشان می گذارند که اینکار ضربه زیادی به ناش

 90%|█████████ | 9/10 [00:09<00:00,  1.17it/s]

article : سیدصالح هندی در گفت وگو با خبرنگار شانا افزود: فاز نخست پروژه های پژوهشی به تدوین نقشه راه مرتبط است که خوشبختانه دانشگاه های طرف قرارداد در این زمینه عملکرد خوبی داشته اند. [n] بر اساس این گزارش، اسفندماه پارسال، شرکت ملی نفت ایران با هدف توسعه زیرساخت های علمی و فناوری در کشف منابع هیدروکربوری، با پنج دانشگاه منتخب کشور شامل دانشگاه شهید بهشتی، فردوسی مشهد، خوارزمی، صنعتی شاهرود و شهید چمران اهواز قرارداد پژوهشی امضا کرد؛ قراردادهایی که مدت زمان اجرای هر یک از آنها ۱۰ سال در نظر گرفته شد. [n] هندی از برگزاری نشست کمیته راهبری در دانشگاه فردوسی مشهد هم خبر داد و عملکرد این دانشگاه را در اجرای پروژه پژوهشی توسعه فناوری بهبود تصویرسازی زیرسطحی در اکتشاف منابع هیدروکربوری در حوضه رسوبی کپه داغ بسیار مطلوب توصیف کرد. [n] وی با اشاره به تاکید شرکت ملی نفت ایران بر اهمیت ایجاد شبکه نوآوری و فناوری گفت: دانشگاه ها در بخش مدیریت فناوری، از مشاوران خوبی بهره گرفته اند و با دانشگاه های داخلی و خارجی، همکاری مناسبی برقرار کرده اند که امیدواریم مجموعه این تلاش ها، به نتایج خوبی برای صنع

100%|██████████| 10/10 [00:10<00:00,  1.06s/it]

article : به گزارش پایگاه خبری اویل پرایس، صنعت حفاری نفت و گاز از زمان افت قیمت های نفت در سال ۲۰۱۴ میلادی بیش از حد با چالش روبه رو شده است. [n] حوزه حفاری و دیگر فعالان این بخش بر خلاف تولید و پالایش، نه تنها به منظور باقی ماندن در بازار رقابت مجبور بوده اند قیمت های خود را به طور چشمگیری کاهش دهند، بلکه با افت شدید سرمایه گذاری ها در بخش اکتشاف روبه رو بوده اند. [n] بنا به گزارش موسسه بین المللی وودمکنزی، این شرایط تا سال آینده میلادی نیز ادامه خواهد یافت. این موسسه تخمین می زند که سرمایه گذاری ها در حوزه اکتشاف نفت و گاز در سال ۲۰۱۷ میلادی افت بیشتری خواهد داشت و به ۳۷ میلیارد دلار خواهد رسید که پایین ترین میزان از سال ۲۰۰۹ میلادی است. [n] این در حالی است که میزان سرمایه گذاری ها در این بخش در سال ۲۰۱۴ میلادی به ۱۰۰ میلیارد دلار رسیده بود. [n] تحلیلگران وودمکنزی اعلام کردند که از سال ۲۰۱۸ میلادی صنعت نفت و گاز رو به بهبود می گذارد، به طوری که میزان سرمایه گذاری در حوزه اکتشاف در سال ۲۰۱۹ میلادی به ۵۰ میلیارد دلار و در سال ۲۰۲۰ میلادی به ۶۰ میلیارد دلار خواهد رسید. [n] این خبر خوبی

In [14]:
article = """
به گزارش گروه دفاعی خبرگزاری تسنیم، سردار سرلشکر محمد باقری رئیس ستادکل نیروهای مسلح  ایران عصر امروز دوشنبه با «آصف علی زرداری» رئیس جمهوری پاکستان دیدار و طرفین در زمینه روابط میان دو کشور همسایه گفتگو کردند.

این بیانیه افزود: طرفین در این دیدار با اشاره به اهمیت روابط طولانی‌ مدت و برادرانه میان پاکستان و ایران بر لزوم تقویت روابط تجاری و اقتصادی فی مابین برای منافع مشترک دو کشور برادر تاکید کردند.

بنا بر اعلام کاخ ریاست جمهوری پاکستان مقامات دو کشور ضمن گفتگو درخصوص اهمیت تقویت همکاری‌های امنیتی و اطلاعاتی میان دو کشور در راستای مبارزه با تروریسم و تهدیدات مشترک اظهار داشتند: همکاری نزدیک ایران و پاکستان در این زمینه می‌تواند به تأمین امنیت و ثبات در مرزهای دو کشور و منطقه کمک کند.

در این بیانیه آمده است: سردار سرلشکر دکتر محمد باقری در دیدار خود با رئیس‌جمهور پاکستان از مواضع مستحکم و حمایتی دولت و ملت این کشور همسایه در قبال حمایت از غزه و لبنان قدردانی کرده است.

سردار باقری که به دعوت فرمانده ارتش پاکستان شب گذشته در یک سفر رسمی دو روزه وارد پاکستان شد، پس از دیدار با مقامات سیاسی و نظامی این کشور از جمله فرمانده ارتش، فرمانده نیروی هوایی ، نخست وزیر پاکستان در مصاحبه ای گفت: هیئت نظامی ایران امروز 5 ملاقات بسیار سودمند با فرمانده ارتش، فرمانده نیروی هوایی، نخست وزیر، رئیس جمهور و اکنون با وزیر دفاع پاکستان داشته است.

وی افزود: در مورد مسائل متعدد صحبت و نتایج سودمندی حاصل شد که یکی از مهم ترین آن مساله ارتقای امنیت مرزهای دو کشور و تبدیل مرزها برای مسیر تجارت و رفاه و آرامش هر دو همسایه بود.

سردار باقری تاکید کرد: نیروهای مسلح ایران و پاکستان درباره مرز و برخورد با تروریست‌ها به طور مفصل بحث و تصمیم گیری شد توسعه روابط دو نیروی مسلح در ابعاد مختلف بررسی شد.

وی اظهار داشت که پاکستان از جمهوری اسلامی ایران برای شرکت در رزمایش چندملیتی امان-25 به میزبانی نیروی دریایی پاکستان دعوت کرد و ما با پذیرش آن  در سطحی که امکان پذیر باشد، شرکت خواهیم کرد.

سرلشکر باقری گفت که درباره موضوعات مورد علاقه در منطقه  از جمله افغانستان بحث مفصلی شد و دو کشور ضرورت هماهنگی بیشتر در این زمینه را مورد تاکید قرار دادند.
"""


encoding = tokenizer.encode_plus(article,max_length =512,padding='max_length',
                                 truncation = True,
                                 return_tensors="pt").to(device)
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

model.eval()
beam_outputs = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=72, # How long the generated summary should be
      early_stopping=True,
      num_beams=5,
      num_return_sequences=1
)

for beam_output in beam_outputs:
    predicted = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)

    print(f"article : {article}")
    print(f"predicted summary : {predicted}")

article : 
به گزارش گروه دفاعی خبرگزاری تسنیم، سردار سرلشکر محمد باقری رئیس ستادکل نیروهای مسلح  ایران عصر امروز دوشنبه با «آصف علی زرداری» رئیس جمهوری پاکستان دیدار و طرفین در زمینه روابط میان دو کشور همسایه گفتگو کردند.

این بیانیه افزود: طرفین در این دیدار با اشاره به اهمیت روابط طولانی‌ مدت و برادرانه میان پاکستان و ایران بر لزوم تقویت روابط تجاری و اقتصادی فی مابین برای منافع مشترک دو کشور برادر تاکید کردند.

بنا بر اعلام کاخ ریاست جمهوری پاکستان مقامات دو کشور ضمن گفتگو درخصوص اهمیت تقویت همکاری‌های امنیتی و اطلاعاتی میان دو کشور در راستای مبارزه با تروریسم و تهدیدات مشترک اظهار داشتند: همکاری نزدیک ایران و پاکستان در این زمینه می‌تواند به تأمین امنیت و ثبات در مرزهای دو کشور و منطقه کمک کند.

در این بیانیه آمده است: سردار سرلشکر دکتر محمد باقری در دیدار خود با رئیس‌جمهور پاکستان از مواضع مستحکم و حمایتی دولت و ملت این کشور همسایه در قبال حمایت از غزه و لبنان قدردانی کرده است.

سردار باقری که به دعوت فرمانده ارتش پاکستان شب گذشته در یک سفر رسمی دو روزه وارد پاکستان شد، پس از دیدا